This notebook will be where we process the columns for our regression model into the best form of themselves.

First, I need to test whether Boolean Remodel or years between build and remodel has more info I will run two regressions, one with each, and keep the column that yields the slightly higher one.

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [4]:
data=pd.read_csv('RowFiltered_dummied_data_TRAIN.csv')

In [5]:
binary_X = data.drop('SalePrice', axis=1).drop('RemodelYrsAftBuilt', axis=1)
YrsAfter_X = data.drop('SalePrice', axis=1).drop('Remodeled', axis=1)
y = data['SalePrice']

In [8]:
ols=LinearRegression()

In [9]:
# Including only Binary remodeled column.
ols.fit(binary_X, y)
ols.score(binary_X, y)

0.9398718274845993

In [10]:
# Including only RemodelYrsAftBuilt column
ols.fit(YrsAfter_X, y)
ols.score(YrsAfter_X, y)

0.9404867901276911

YrsAfter gives a better $R^2$. Now, lets make sure its multicollinearity with the other columns is of a manageable level.

In [11]:
# To test the YrsAfter column:
y = data['RemodelYrsAftBuilt']
colX = YrsAfter_X = data.drop('SalePrice', axis=1).drop('Remodeled', axis=1).drop('RemodelYrsAftBuilt', axis=1)
ols.fit(colX, y)
ols.score(colX, y)

0.7373052059575511

It is very multicollinear, disappointing. Lets check what it was WITH the boolean column, to see if we at least reduce it by eliminating the boolean.

In [12]:
# To test the YrsAfter column:
y = data['RemodelYrsAftBuilt']
colX = YrsAfter_X = data.drop('SalePrice', axis=1).drop('RemodelYrsAftBuilt', axis=1)
ols.fit(colX, y)
ols.score(colX, y)

0.773157050029327

Yeah, thats a pretty decent reduction in multicollinearity from the removal of just one column in 100. Going forward we will use the RemodelYrsAftBuilt column.

## Feature Importance for LndAc_S

In [15]:
train = data.copy()
test = pd.read_csv('RowFiltered_dummied_data_TEST.csv')

trainX = train.drop('SalePrice', axis=1)
train_y = train['SalePrice']
testX = test.drop('SalePrice', axis=1)
test_y = test['SalePrice']



In [16]:
rcols=["MasVnrType_BrkFace" ,  "Foundation_Stone" ,    "Exterior1st_AsbShng" , "LotArea"             
 , "Fireplaces"    ,       "HeatingQC_TA"     ,    "Exterior1st_BrkFace",  "Foundation_Wood"     
 , "Condition1_Artery" ,   "BsmtFinGdLvng"    ,    "Functional_Min" ,      "Neighborhood_NoRidge"
,"ExterQual_Gd"     ,    "BsmtExposure"       ,  "KitchenQual_Ex"  ,     "BsmtQual_Ex"         
,"Functional_Maj"    ,   "OverallCondBinary"  ,  "OverallQual"  ,        "ExterQual_Ex"        
,"TotalBsmtSF"   ,       "BsmtUnfSF"         ,   "GrLivArea"      ,      "LndAc_S" ]
data_trainxsubsetR = trainX[rcols]
data_testxsubsetR = testX[rcols]
lasso.set_params(alpha=0)
lasso.fit(data_trainxsubsetR, train_y) 
(lasso.score(data_trainxsubsetR, train_y))
(lasso.score(data_testxsubsetR, test_y))

NameError: name 'lasso' is not defined

In [38]:
stdev = np.sqrt(sum((lasso.predict(data_trainxsubsetR) - data_trainy)**2) / (len(data_trainy) - 2))
predictiondf=pd.DataFrame({'Prediction': lasso.predict(data_testxsubsetR), 'Lower Bound 95%CI': lasso.predict(data_testxsubsetR)- 1.96*stdev,'Upper Bound 95%CI':lasso.predict(data_testxsubsetR)+ 1.96*stdev})

actualpredictions=lasso.predict(data_trainxsubsetR)
# finding avrg mse for prediction in +-10% price range
avrgmseperpredict=predictiondf.Prediction.apply(lambda x: mseprediction(x, actualpredictions, data_trainy))

predictiondf=pd.DataFrame({'Prediction': lasso.predict(data_testxsubsetR),'Lower Bound Avrg MSE':-avrgmseperpredict+ lasso.predict(data_testxsubsetR),'Upper Bound Avrg MSE':avrgmseperpredict+ lasso.predict(data_testxsubsetR),'Lower Bound 95%CI': lasso.predict(data_testxsubsetR)- 1.96*stdev,'Upper Bound 95%CI':lasso.predict(data_testxsubsetR)+ 1.96*stdev})

predictiondf




,Prediction,Lower Bound Avrg MSE,Upper Bound Avrg MSE,Lower Bound 95%CI,Upper Bound 95%CI
0,184309.285059,166235.193957,202383.376160,146981.385892,221637.184225
1,99041.798399,82747.133475,115336.463323,61713.899233,136369.697566
2,147320.847727,132906.633827,161735.061628,109992.948561,184648.746894
3,474512.155522,406208.080981,542816.230063,437184.256355,511840.054689
4,233961.230350,212964.612631,254957.848070,196633.331184,271289.129517
...,...,...,...,...,...
847,94210.793085,77622.605814,110798.980356,56882.893919,131538.692252
848,98800.174745,82535.340102,115065.009389,61472.275579,136128.073912
849,140156.412143,126155.747498,154157.076789,102828.512977,177484.311310
850,138329.659243,124252.033517,152407.284969,101001.760077,175657.558410
